### import packages

In [1]:
from utils import data_scraping
import pandas as pd
import os
from googletrans import Translator #googletrans==4.0.0rc1
import aspect_based_sentiment_analysis as absa #aspect-based-sentiment-analysis
import spacy

2023-08-22 11:57:45.536389: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 11:57:45.580999: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-22 11:57:45.582104: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 11:57:46.321570: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/tdjiang/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### read & transform data

In [187]:
# read data
file_name = 'data_the_grace_dalat_reviews.csv'
if os.path.exists(file_name):
    the_grace_dalat_reviews = pd.read_csv(file_name, converters={'profileId':str})
    the_grace_dalat_reviews[['travelPurpose','travelKeywords','photoDataDisplaysList','reactionSummaries']] = the_grace_dalat_reviews[['travelPurpose','travelKeywords','photoDataDisplaysList','reactionSummaries']].applymap(lambda x: eval(x) if isinstance(x, str) else x)
else:
    the_grace_dalat_scraping = data_scraping(url_hotel="https://www.traveloka.com/vi-vn/hotel/vietnam/the-grace-hotel-dalat-3000010042556", reviews_per_page=10)
    the_grace_dalat_reviews = the_grace_dalat_scraping.get_all_reviews()
    the_grace_dalat_reviews.to_csv(file_name, index=False)

In [188]:
# replace empty data to blank data
the_grace_dalat_reviews = the_grace_dalat_reviews.mask(the_grace_dalat_reviews == '')

# drop columns with no data
the_grace_dalat_reviews = the_grace_dalat_reviews.dropna(axis=1, how='all')

# add new columns
the_grace_dalat_reviews['travelPurposeText'] = the_grace_dalat_reviews.travelPurpose.apply(lambda x: x['travelPurposeText'] if isinstance(x, dict) else x)
the_grace_dalat_reviews['travelPurpose'] = the_grace_dalat_reviews.travelPurpose.apply(lambda x: x['travelPurpose'] if isinstance(x, dict) else x)
the_grace_dalat_reviews['travelKeywords'] = the_grace_dalat_reviews.travelKeywords.apply(lambda x: ','.join(sorted(map(lambda y: y['travelKeyword'] if isinstance(y, dict) else '', x))) if isinstance(x, list) else x)
the_grace_dalat_reviews['reviewLikes'] = the_grace_dalat_reviews.reactionSummaries.apply(lambda x: x['reactionSummaryMap']['LIKE']['reactionCount'] if isinstance(x, dict) else x)
the_grace_dalat_reviews['photoCategories'] = the_grace_dalat_reviews.photoDataDisplaysList.apply(lambda x: ','.join(sorted(map(lambda y: y['photoCategoryDisplay']['photoCategory'] if isinstance(y, dict) else '', x))) if isinstance(x, list) else x)

In [190]:
the_grace_dalat_reviews[['reviewId','timestamp','profileId','reviewerName','overallScore','cleanlinessScore','comfortScore','serviceScore','foodScore','locationScore',
                         'originalReviewText','reviewText','travelThemeDisplayName','travelPurpose','travelKeywords','language','translated','reviewLikes','photoCategories','travelPurposeText']].head(2)

,reviewId,timestamp,profileId,reviewerName,overallScore,cleanlinessScore,comfortScore,serviceScore,foodScore,locationScore,originalReviewText,reviewText,travelThemeDisplayName,travelPurpose,travelKeywords,language,translated,reviewLikes,photoCategories,travelPurposeText
0,127452257,1684722833270,224137865,Hoang N.,9.7,10.0,10.0,10.0,NaN,10.0,"Khách sạn rất xinh , vượt ngoài mong đợi của m...","The hotel is very beautiful, beyond my expecta...",Liburan,LEISURE,FRIENDS,ENGLISH,True,2,"BEDROOM,BEDROOM",Liburan
1,126831568,1660817392070,247430533,Thuy Q. N.,9.7,10.0,10.0,8.0,10.0,10.0,Phòng ốc sạch sẽ đầy đủ tiện nghi đáng để lưu trú,Clean rooms with full amenities worth to stay,Staycation,STAYCATION,FAMILY,ENGLISH,True,6,BEDROOM,Staycation
